In [ ]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
from ipywidgets import interact
from google.colab import drive
from scipy.io import loadmat
import requests

drive.mount('/content/gdrive')




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd '/content/gdrive/Shared drives/GCPDS - semillero/databases/music_bci/'

/content/gdrive/Shared drives/GCPDS - semillero/databases/music_bci


In [ ]:
S1=loadmat('VPaak.mat')
X=S1['data'][0][0][0].T
fs=S1['data']['fs'][0][0][0][0]
channel=55
N_max=1000                         #Se escoje canal y se recorta tiempo
data=X[channel][0:N_max]



def butter(data,lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq 
    high = highcut / nyq
    b, a = sig.butter(order, [low, high], btype='band')
    y = sig.lfilter(b, a, data)
    w, h = sig.freqz(b, a)
    return y,w,h


def chebyshev(data,lowcut, highcut, fs, order,rp):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = sig.cheby1(order, rp,[low, high], btype='band')
    y = sig.lfilter(b, a, data)
    w, h = sig.freqz(b, a)
    return y,w,h

def bessel(data,lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = sig.bessel(order, [low, high], btype='band')
    y = sig.lfilter(b, a, data)
    w, h = sig.freqz(b, a)
    return y,w,h
  

def eliptic(data,lowcut, highcut, fs, order,rp,rs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = sig.ellip(order, rp, rs, [low, high], btype='band')
    y = sig.lfilter(b, a, data)
    w, h = sig.freqz(b, a)
    return y,w,h

def plot_sub(data,title,pos):
  plt.subplot(2,2,pos)
  plt.plot(data)
  plt.grid()
  plt.title(title)

def plot(data,title):
  plt.figure(figsize=(20,2))
  plt.plot(data)
  plt.grid()
  plt.title(title)

def time_plot(data1,data2,data3,data4,title1,title2,title3,title4):
  plt.figure(figsize=(20,3))
  plot_sub(data1,title1,1)
  plot_sub(data2,title2,2)
  plot_sub(data3,title3,3)
  plot_sub(data4,title4,4)

def freq_plot(w1,w2,w3,w4,data1,data2,data3,data4,title1,title2,title3,title4,low,high):
  plt.figure(figsize=(20,5))

  plt.subplot(2,2,1)
  plt.semilogx((0.5*fs/np.pi)*w1,20*np.log10(np.abs(data1)))
  plt.title(title1)
  plt.axvline(low, color='green')  
  plt.axvline(high, color='red')  
  #plt.xlim(0.75*low,1.5*high)  
  plt.grid()

  plt.subplot(2,2,2)
  plt.semilogx((0.5*fs/np.pi)*w2,20*np.log(np.abs(data2)))
  plt.title(title2)
  plt.axvline(low, color='green')  
  plt.axvline(high, color='red') 
  #plt.xlim(0.75*low,1.5*high)
  plt.grid()

  plt.subplot(2,2,3)
  plt.semilogx((0.5*fs/np.pi)*w3,20*np.log10(np.abs(data3)))
  plt.title(title3)
  plt.axvline(low, color='green')  
  plt.axvline(high, color='red') 
  #plt.xlim(0.75*low,1.5*high)
  plt.grid()

  plt.subplot(2,2,4)
  plt.semilogx((0.5*fs/np.pi)*w4,20*np.log10(np.abs(data4)))
  plt.title(title4)
  plt.axvline(low, color='green')  
  plt.axvline(high, color='red') 
  #plt.xlim(0.75*low,1.5*high)
  plt.grid()


In [ ]:
@interact(N=(1,10,1),low_cut=(2,11.5,0.5),high_cut=(12,20,0.5))
def master(N=3,low_cut=8,high_cut=12):  
  fs=200
  #low_cut=8
  #high_cut=12
  #N=3
  data=np.random.randn(1000)  
  plot(data,'Original')

  bu,wu,hu=butter(data,low_cut,high_cut,fs,N)
  ch,wc,hc=chebyshev(data,low_cut,high_cut,fs,N,1)   #Respuestas en frecuencia y tiempo de los 4 filtros
  be,wbe,hbe=bessel(data,low_cut,high_cut,fs,N)
  el,we,he=eliptic(data,low_cut,high_cut,fs,N,1,40)  

  time_plot(bu,ch,be,el,'Butterworth','Chebyshev','Bessel','Elliptic')
  freq_plot(wu,wc,wbe,we,hu,hc,hbe,he,'Butterworth','Chebyshev','Bessel','Elliptic',low_cut,high_cut)








interactive(children=(IntSlider(value=3, description='N', max=10, min=1), FloatSlider(value=8.0, description='…